In [2]:
regressors_to_remove = ["destinationMacAddress", "sourceMacAddress", "egressInterface", "ingressInterface", "initialTCPFlags", 
                    "reverseInitialTCPFlags", "reverseTcpUrgTotalCount", "reverseUnionTCPFlags", "silkAppLabel", 
                    "tcpSequenceNumber", "tcpUrgTotalCount", "unionTCPFlags", "vlanId", "sourceIPv4Address", 
                    "destinationIPv4Address", "reverseTcpSequenceNumber", "observationDomainId", "reverseStandardDeviationInterarrivalTime",
                    "reverseStandardDeviationPayloadLength", "reverseSmallPacketCount", "reverseNonEmptyPacketCount",
                    "reverseMaxPacketSize", "reverseLargePacketCount", "reverseFirstNonEmptyPacketSize", "reverseDataByteCount",
                    "reverseBytesPerPacket", "reverseAverageInterarrivalTime", "collectorName"]

regressor_has_null = ["protocolIdentifier"]

This code will just read in the downloaded test dataset

In [1]:
import pandas as pd
df1 = pd.read_json('train_df_noGoogle.json', lines=True)
df2 = pd.read_json('train_google.json', lines=True)
train_df = pd.concat([df1,df2],ignore_index=True)
test_df = pd.read_json('test_df.json', lines=True)

This code will individually read the raw json data and create a train response

In [3]:
import os
import re
import json
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.sql.functions import *
import gc
#spark = SparkSession.builder.appName("cleanData").getOrCreate()



folder_path = 'C:\\Users\\Priyash Shah\\Downloads\\DATA3001 Project Data\\Extracted Data'
data_lengths = []
# Get all files in the directory
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

#df = spark.createDataFrame(spark.sparkContext.emptyRDD(),schema)
train_df = pd.DataFrame()
for file in files:
    
    file_path = folder_path+'\\'+file
    data = []
    length = 0
    ### TEMP
    dataset = open(file_path)
    print(f"in file {file}")
    # Read file line by line based on date
    for line in dataset:
        obj = json.loads(line)
        
        if 'flows' in obj:
           
            dateflow = obj['flows']['flowEndMilliseconds']
            if dateflow > "2019-08-26":
                break
            length += 1    
            data.append(obj['flows'])
        else:
            break
    print(f"part 1")
    response = re.sub('([^.]*).json$', r'\1', file)
    data_info = (response, length)        
    data_lengths.append(data_info)
    print(f"part 2")
    # Assuming data is already loaded
    temp_df = pd.DataFrame(data)
    # Removing regressors we are not interested in
    print(f"part 3")
    for regressors in regressors_to_remove:
        if regressors in temp_df.columns: 
            temp_df = temp_df.drop(columns=regressors)
    print(f"part 4")

    # Sampling
    temp_df = temp_df.dropna().sample(n = 5000, random_state=69)
    print(f"part 5")
    # print(temp_df.columns)
    # Dropping columns
    
    # Dropping rows with all NaN values in the specified columns
    #for regressors in regressor_has_null:
    #    temp_df = temp_df.dropna(how='all', subset=[regressors])
    temp_df['response'] = response    
    train_df = pd.concat([train_df, temp_df], ignore_index=True)
    # Explicitly delete temp_df and free up memory
    del temp_df
    gc.collect()


in file amazon_echo_gen2.json
part 1
part 2
part 3
part 4
part 5
in file au_network_camera.json
part 1
part 2
part 3
part 4
part 5
in file au_wireless_adapter.json
part 1
part 2
part 3
part 4
part 5
in file bitfinder_awair_breathe_easy.json
part 1
part 2
part 3
part 4
part 5
in file candy_house_sesami_wi-fi_access_point.json
part 1
part 2
part 3
part 4
part 5
in file i-o_data_qwatch.json
part 1
part 2
part 3
part 4
part 5
in file irobot_roomba.json
part 1
part 2
part 3
part 4
part 5
in file jvc_kenwood_cu-hb1.json
part 1
part 2
part 3
part 4
part 5
in file jvc_kenwood_hdtv_ip_camera.json
part 1
part 2
part 3
part 4
part 5
in file line_clova_wave.json
part 1
part 2
part 3
part 4
part 5
in file link_japan_eremote.json
part 1
part 2
part 3
part 4
part 5
in file mouse_computer_room_hub.json
part 1
part 2
part 3
part 4
part 5
in file nature_remo.json
part 1
part 2
part 3
part 4
part 5
in file panasonic_doorphone.json
part 1
part 2
part 3
part 4
part 5
in file philips_hue_bridge.json
part 1


Does the same as above but for test

In [3]:
import os
import re
import json
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.sql.functions import *
import gc

folder_path = 'C:\\Users\\Priyash Shah\\Downloads\\DATA3001 Project Data\\Extracted Data'
data_lengths = []
# Get all files in the directory
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

#df = spark.createDataFrame(spark.sparkContext.emptyRDD(),schema)
test_df = pd.DataFrame()
for file in files:
    print(f"in file {file}")
    file_path = folder_path+'\\'+file
    data = []
    length = 0
    ### TEMP
    dataset = open(file_path)
    
    # Read file line by line based on date
    for line in dataset:
        obj = json.loads(line)
        
        if 'flows' in obj:
           
            dateflow = obj['flows']['flowEndMilliseconds']
            if dateflow > "2019-09-10":
                break
            if dateflow > "2019-08-27":
                length += 1    
                data.append(obj['flows'])
            
        else:
            break
    
    response = re.sub('([^.]*).json$', r'\1', file)
    data_info = (response, length)        
    data_lengths.append(data_info)
    
    # Assuming data is already loaded
    temp_df = pd.DataFrame(data)
    
    # Removing regressors we are not interested in
    for regressors in regressors_to_remove:
        if regressors in temp_df.columns: 
            temp_df = temp_df.drop(columns=regressors)

    # Sampling
    temp_df = temp_df.dropna().sample(n = 1000, random_state=69)

    # print(temp_df.columns)
    # Dropping columns
    
    # Dropping rows with all NaN values in the specified columns
    for regressors in regressor_has_null:
        temp_df = temp_df.dropna(how='all', subset=[regressors])
    temp_df['response'] = response    
    test_df = pd.concat([test_df, temp_df], ignore_index=True)
            


in file amazon_echo_gen2.json
in file au_network_camera.json
in file au_wireless_adapter.json
in file bitfinder_awair_breathe_easy.json
in file candy_house_sesami_wi-fi_access_point.json
in file google_home_gen1.json
in file i-o_data_qwatch.json
in file irobot_roomba.json
in file jvc_kenwood_cu-hb1.json
in file jvc_kenwood_hdtv_ip_camera.json
in file line_clova_wave.json
in file link_japan_eremote.json
in file mouse_computer_room_hub.json
in file nature_remo.json
in file panasonic_doorphone.json
in file philips_hue_bridge.json
in file planex_camera_one_shot!.json
in file planex_smacam_outdoor.json
in file planex_smacam_pantilt.json
in file powerelectric_wi-fi_plug.json
in file qrio_hub.json
in file sony_bravia.json
in file sony_network_camera.json
in file sony_smart_speaker.json
in file xiaomi_mijia_led.json


In [118]:
test_df

,flowDurationMilliseconds,reverseFlowDeltaMilliseconds,protocolIdentifier,sourceTransportPort,packetTotalCount,octetTotalCount,flowAttributes,destinationTransportPort,reversePacketTotalCount,reverseOctetTotalCount,...,nonEmptyPacketCount,dataByteCount,averageInterarrivalTime,firstNonEmptyPacketSize,largePacketCount,maxPacketSize,standardDeviationPayloadLength,standardDeviationInterarrivalTime,bytesPerPacket,response
0,0.009,0.009,2,53406,1,71,0,53,1,139,...,1,43,0,43,0,43,0,0,43,0
1,0.003,0.000,1,47580,2,111,0,443,0,0,...,1,31,3,31,0,31,0,0,31,0
2,0.036,0.000,0,8,10,840,1,0,0,0,...,10,560,4,56,0,56,0,2,56,0
3,0.010,0.010,2,42680,1,75,0,53,1,139,...,1,47,0,47,0,47,0,0,47,0
4,0.030,0.030,2,17525,1,71,0,53,1,139,...,1,43,0,43,0,43,0,0,43,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0.576,0.000,2,40445,2,156,1,137,0,0,...,2,100,576,50,0,50,0,0,50,24
24996,1785.302,0.118,1,456,242,14144,0,80,124,9072,...,121,4464,7407,32,1,624,4,7213,36,24
24997,0.578,0.000,2,57819,2,156,1,137,0,0,...,2,100,578,50,0,50,0,0,50,24
24998,0.584,0.000,2,41411,2,156,1,137,0,0,...,2,100,584,50,0,50,0,0,50,24


Cleans the data for XGboost

In [2]:
columns_to_remove = ["flowStartMilliseconds", "flowEndMilliseconds",'firstEightNonEmptyPacketDirections']
for regressors in columns_to_remove:
    if regressors in train_df.columns:
        train_df = train_df.drop(columns=regressors)
        test_df = test_df.drop(columns=regressors)


In [3]:
# Encoding the response to numeric values

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_df['response'] = le.fit_transform(train_df['response'])
test_df['response'] = le.fit_transform(test_df['response'])

In [4]:
# Converting Variables into category
columns_categorical = ["flowAttributes", "protocolIdentifier", "ipClassOfService", "flowEndReason",
                      'reverseFlowAttributes']


for regressors in columns_categorical:
    train_df[regressors] = train_df[regressors].astype('category')
    test_df[regressors] = test_df[regressors].astype('category')


In [5]:
for col in columns_categorical:
    train_df[col] = train_df[col].cat.codes
    test_df[col] = test_df[col].cat.codes


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125000 entries, 0 to 124999
Data columns (total 24 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   flowDurationMilliseconds           125000 non-null  float64
 1   reverseFlowDeltaMilliseconds       125000 non-null  float64
 2   protocolIdentifier                 125000 non-null  int8   
 3   sourceTransportPort                125000 non-null  int64  
 4   packetTotalCount                   125000 non-null  int64  
 5   octetTotalCount                    125000 non-null  int64  
 6   flowAttributes                     125000 non-null  int8   
 7   destinationTransportPort           125000 non-null  int64  
 8   reversePacketTotalCount            125000 non-null  int64  
 9   reverseOctetTotalCount             125000 non-null  int64  
 10  reverseFlowAttributes              125000 non-null  int8   
 11  ipClassOfService                   1250

### XG Boost

In [7]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

response = ['response']
predictors = [x for x in list(train_df.columns) if x not in response]

X = train_df[predictors]
y = train_df[response]
X_test = test_df[predictors]
y_test = test_df[response]
#label = train_df['response']
#dtrain = xgb.DMatrix(train_df, label=label, enable_categorical=True)
# Instantiate an XGBClassifier
model = xgb.XGBClassifier(objective ='multi:softprob', 
                          num_class = 19,
                          colsample_bytree = 0.3, 
                          learning_rate = 0.1,
                          max_depth = 10, 
                          alpha = 1, 
                          n_estimators = 50)

# Train the model
model.fit(X, y)

# Predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.4118


### Exploring prediction errors

In [82]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

y_test_arr = np.ravel(y_test)
# Step 2: Create a DataFrame with true labels and predicted labels
results_df = pd.DataFrame({'True': y_test_arr, 'Predicted': y_pred})

# Step 3: Compare the true and predicted labels to find mismatches
results_df['Correct'] = results_df['True'] == results_df['Predicted']

# Step 4: Extract indices of incorrectly classified instances
incorrect_indices = results_df[results_df['Correct'] == False].index.tolist()

# Now `incorrect_indices` holds the indices of the incorrectly classified instances
#print("Indices of incorrectly classified instances:", incorrect_indices)




In [9]:
row = 0
prediction_int = y_test.iloc[row]
print(f'It should be {prediction_int}, I am getting {y_pred[row]}')

It should be response    0
Name: 0, dtype: int32, I am getting 0


In [10]:
probabilities = model.predict_proba(X_test)
probabilities

array([[0.52821815, 0.00714703, 0.03455031, ..., 0.00629136, 0.01075714,
        0.00702013],
       [0.7807144 , 0.00256663, 0.00435964, ..., 0.00200704, 0.00305754,
        0.00264425],
       [0.91762847, 0.00104874, 0.00098997, ..., 0.00118892, 0.00112028,
        0.00098989],
       ...,
       [0.02109229, 0.03403796, 0.01524978, ..., 0.04164699, 0.01014183,
        0.03341097],
       [0.02103613, 0.02710547, 0.01372556, ..., 0.04248632, 0.0093953 ,
        0.04124414],
       [0.01933529, 0.02646217, 0.01408833, ..., 0.04258097, 0.00934198,
        0.03754492]], dtype=float32)

In [11]:
probabilities[row,:]

array([0.52821815, 0.00714703, 0.03455031, 0.00774228, 0.00679514,
       0.20508993, 0.00740229, 0.00716077, 0.05794402, 0.00720267,
       0.009235  , 0.00659496, 0.00622814, 0.0069524 , 0.00965795,
       0.00919403, 0.00615403, 0.0070845 , 0.01066438, 0.00879017,
       0.00732048, 0.0188028 , 0.00629136, 0.01075714, 0.00702013],
      dtype=float32)

In [44]:
## Hyper parameter tuning
model = xgb.XGBClassifier(use_label_encoder=False)
param_grid = {
    'n_estimators': [50,100],
    'learning_rate': [0.1,0.2],
    'max_depth': [5,10],
    'alpha': [1,2]
}
model = xgb.XGBClassifier(use_label_encoder=False)


In [45]:
grid_search = GridSearchCV(model, param_grid, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)
grid_search.fit(X, y)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'alpha': [1, 2], 'learning_rate': [0.1, 0.2],
                         'max_depth': [5, 10], 'n_estimators': [50, 100]},
             scoring='accuracy', verbose=1)

In [46]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Best hyperparameters:", grid_search.best_params_)
print("Test accuracy:", accuracy_score(y_test, y_pred))

Best hyperparameters: {'alpha': 2, 'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 50}
Test accuracy: 0.4669473684210526


In [30]:
import numpy as np
conversion = np.arange(0,25)
decoded_category = le.inverse_transform(conversion)
for i in conversion:
    print(f'{i} = {decoded_category[i]}')

0 = amazon_echo_gen2
1 = au_network_camera
2 = au_wireless_adapter
3 = bitfinder_awair_breathe_easy
4 = candy_house_sesami_wi-fi_access_point
5 = google_home_gen1
6 = i-o_data_qwatch
7 = irobot_roomba
8 = jvc_kenwood_cu-hb1
9 = jvc_kenwood_hdtv_ip_camera
10 = line_clova_wave
11 = link_japan_eremote
12 = mouse_computer_room_hub
13 = nature_remo
14 = panasonic_doorphone
15 = philips_hue_bridge
16 = planex_camera_one_shot!
17 = planex_smacam_outdoor
18 = planex_smacam_pantilt
19 = powerelectric_wi-fi_plug
20 = qrio_hub
21 = sony_bravia
22 = sony_network_camera
23 = sony_smart_speaker
24 = xiaomi_mijia_led


In [13]:
tot = 0
between = 1000
i = 0
print(f"Gradient Boosted Decision Tree Accuracy: {accuracy*100}%\n \nclass by class:")
while(between <= 25000):
    unique, counts = np.unique(y_pred[between-1000:between], return_counts=True)
    i_count = np.sum(y_pred[between-1000:between] == i)
    tot = np.sum(y_pred[:] == i)
    #tot += i_count
    percentage = i_count
    print(f'{decoded_category[i]}, accuracy = {percentage}%')
    #print(f'{i} = {decoded_category[i]}, we are accurate {percentage}% amount of times\n{dict(zip(unique, counts))}')
    between += 1000
    i += 1

Gradient Boosted Decision Tree Accuracy: 41.18%
 
class by class:
amazon_echo_gen2, accuracy = 652%
au_network_camera, accuracy = 112%
au_wireless_adapter, accuracy = 731%
bitfinder_awair_breathe_easy, accuracy = 152%
candy_house_sesami_wi-fi_access_point, accuracy = 49%
google_home_gen1, accuracy = 978%
i-o_data_qwatch, accuracy = 935%
irobot_roomba, accuracy = 60%
jvc_kenwood_cu-hb1, accuracy = 811%
jvc_kenwood_hdtv_ip_camera, accuracy = 95%
line_clova_wave, accuracy = 361%
link_japan_eremote, accuracy = 40%
mouse_computer_room_hub, accuracy = 992%
nature_remo, accuracy = 1%
panasonic_doorphone, accuracy = 880%
philips_hue_bridge, accuracy = 255%
planex_camera_one_shot!, accuracy = 984%
planex_smacam_outdoor, accuracy = 105%
planex_smacam_pantilt, accuracy = 93%
powerelectric_wi-fi_plug, accuracy = 551%
qrio_hub, accuracy = 249%
sony_bravia, accuracy = 896%
sony_network_camera, accuracy = 146%
sony_smart_speaker, accuracy = 104%
xiaomi_mijia_led, accuracy = 63%


### Threshold without standardising

In [17]:

# Get class probabilities
probabilities = model.predict_proba(X_test)

# Apply threshold
# This will create a 2D array of boolean values where True indicates
# that the corresponding probability exceeds the threshold
threshold_met = probabilities > 0.5

# Create an array to store your predictions with a default value for unclassified
# Assuming -1 is the marker for unclassified instances
predictions = np.full((probabilities.shape[0],), -1)

for i, instance in enumerate(threshold_met):
    # Check if any probability meets the threshold
    if any(instance):
        # Get the index of the max probability above the threshold
        predictions[i] = np.argmax(probabilities[i])

# Now 'predictions' contains the class with the highest probability above the threshold
# or -1 for those where no probability exceeded the threshold.


### Threshold with standardising

In [69]:
### standardising the probabilities based on individual class means and std deviations
import pandas as pd
probabilities = model.predict_proba(X_test)
prob = pd.DataFrame(probabilities)
mean_prob = prob.mean(axis=0)
std_deviation = prob.std(axis=0)

mean_prob = np.ravel(mean_prob)
std_deviation = np.ravel(std_deviation)
mean_reduced_prob = (probabilities - mean_prob) / std_deviation

In [70]:



z_score_50_percent = (0.5 - mean_prob) / std_deviation
threshold = mean_reduced_prob > z_score_50_percent
predictions = np.full((probabilities.shape[0],), -1)
for i, instance in enumerate(threshold):
    # Check if any probability meets the threshold
    if any(instance):
        # Get the index of the max probability above the threshold
        predictions[i] = np.argmax(mean_reduced_prob[i])


In [71]:
dropped = (predictions == -1).sum()
total = len(predictions)
print(f'dropped: {dropped}, total = {total}, percent = {dropped/total*100}%')

dropped: 14366, total = 25000, percent = 57.464000000000006%


In [76]:


y_test_arr = np.ravel(y_test)
# Step 2: Create a DataFrame with true labels and predicted labels
results_df = pd.DataFrame({'True': y_test_arr, 'Predicted': predictions})

classified = results_df['Predicted'] != -1

prediction_cert = results_df[classified]

prediction_cert['Correct'] = prediction_cert['True'] == prediction_cert['Predicted']
true_count = prediction_cert['Correct'].sum()
true_count

C:\Users\Priyash Shah\AppData\Local\Temp\ipykernel_12704\3339173291.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction_cert['Correct'] = prediction_cert['True'] == prediction_cert['Predicted']


10058

In [81]:
prediction_cert.shape

(10634, 3)

In [79]:
10058/10634

0.9458341169832613

### Class by class accuracy

In [75]:
import numpy as np
conversion = np.arange(0,25)
decoded_category = le.inverse_transform(conversion)
y_test_arr = np.ravel(y_test)

def classByClassAccuracy(predictions, actual):
    ind = 0
    classes = {}
    for j in predictions:
        if j == -1:
            i = 'failed'
        else: 
            i = decoded_category[j]
        if i in classes.keys():
            
            if i == decoded_category[actual[ind]]:
                classes[i]["count"] += 1
            classes[i]["total"] += 1
        else:
            
            if i == decoded_category[actual[ind]]:
                classes[i] = {"count":1, "total":1}
            else:
                classes[i] = {"count":0, "total":1}
        
        ind += 1
 
    results = {}
    for k in classes.keys():
        results[k] = (classes[k]["count"], classes[k]["total"])
 
    return dict(sorted(results.items(), key=lambda x:x[1], reverse=True))
class_acc = classByClassAccuracy(predictions, y_test_arr)
class_acc

{'mouse_computer_room_hub': (992, 992),
 'planex_camera_one_shot!': (984, 984),
 'i-o_data_qwatch': (931, 937),
 'sony_bravia': (885, 900),
 'panasonic_doorphone': (849, 850),
 'jvc_kenwood_cu-hb1': (795, 797),
 'au_wireless_adapter': (723, 723),
 'amazon_echo_gen2': (625, 627),
 'google_home_gen1': (573, 855),
 'powerelectric_wi-fi_plug': (549, 552),
 'sony_smart_speaker': (541, 788),
 'line_clova_wave': (302, 303),
 'qrio_hub': (239, 239),
 'philips_hue_bridge': (236, 244),
 'bitfinder_awair_breathe_easy': (149, 149),
 'sony_network_camera': (146, 146),
 'au_network_camera': (108, 110),
 'planex_smacam_outdoor': (105, 105),
 'planex_smacam_pantilt': (86, 86),
 'jvc_kenwood_hdtv_ip_camera': (78, 80),
 'candy_house_sesami_wi-fi_access_point': (46, 46),
 'link_japan_eremote': (41, 45),
 'irobot_roomba': (39, 40),
 'xiaomi_mijia_led': (36, 36),
 'failed': (0, 14366)}

In [45]:
i = 0
for j in predictions:
    i += 1
i

25000

### RANDOM FOREST

In [42]:
# Converting Variables into category
columns_categorical = ["flowAttributes", "protocolIdentifier", "ipClassOfService", "flowEndReason",
                      'reverseFlowAttributes']


for regressors in columns_categorical:
    train_df = pd.get_dummies(train_df, columns=[regressors], prefix=regressors)
    test_df = pd.get_dummies(test_df, columns=[regressors], prefix=regressors)



KeyError: "None of [Index(['flowAttributes'], dtype='object')] are in the [columns]"

In [ ]:
# Automatically remove inconsistent columns/predictors

for regressors in test_df.columns:
    if regressors not in train_df.columns:
         test_df = test_df.drop(columns=regressors)

In [45]:
# Manually

category = ['ipClassOfService_0xd0']
for regressors in category:
    if regressors in train_df.columns:
        train_df = train_df.drop(columns=regressors)
    if regressors in test_df.columns:
        test_df = test_df.drop(columns=regressors)


In [52]:
from sklearn.ensemble import RandomForestClassifier

response = ['response']
predictors = [x for x in list(train_df.columns) if x not in response]

X = train_df[predictors]
y = train_df[response]
X_test = test_df[predictors]
y_test = test_df[response]
forest1 = RandomForestClassifier(criterion='entropy', bootstrap=True, n_estimators = 400)
forest1 = forest1.fit(X, y)
res = forest1.predict(X_test)



C:\Users\Priyash Shah\Downloads\Anaconda\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [57]:
y_test['response'][0]

'amazon_echo_gen2'

In [60]:
from sklearn.metrics import accuracy_score
accuracies = accuracy_score(y_test, res)
count = 0
total = 0
for k in res:
    if k == y_test['response'][total]:
        count +=1
    total += 1

print(f"sklearn: {accuracies}\nme:      {count/total}")

sklearn: 0.44042105263157894
me:      0.44042105263157894
